# Configuração

Instala as libs que serão utilizadas na análise dos dados

In [1]:
!pip install --upgrade feather-format -q
!pip install --upgrade pandas -q
!pip install --upgrade seaborn -q
!pip install --upgrade spotipy -q

    100% |████████████████████████████████| 12.5MB 2.9MB/s 
    100% |████████████████████████████████| 10.1MB 3.7MB/s 
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.2 which is incompatible.
    100% |████████████████████████████████| 215kB 25.4MB/s 


# Extração

### Importações de bibliotecas iniciais

In [0]:
import requests
from requests.exceptions import BaseHTTPError
import io
import pandas as pd
import datetime
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data

### Função que recebe o link para recuperar os dados e retorna um data frame

Foi criado o tipo pois para o chart a primeira linha era o cabeçalho, porém para o viral não. Dessa forma a fim de utilizar a mesma função para ambos os casos foi criado o parâmetro que recebe 1 para chart e 2 para viral.

In [0]:
def pandazize(date, tipo='chart'):
  # verifica se é tipo = 1 (chart)
  if tipo == 'chart':
    link = 'https://spotifycharts.com/regional/br/daily/' + str(date)[:10] + '/download'
    header = 1
  elif tipo == 'viral':
    link = 'https://spotifycharts.com/viral/br/daily/' + str(date)[:10] + '/download'
    header = 0
  else:
    raise ValueError(tipo)
  response = requests.get(link)
  if response.ok:
    file_object = io.StringIO(response.content.decode('utf-8'))
    return pd.read_csv(file_object, header=header)
  else:
    return pd.DataFrame()#@RUI NÃO MEXE NESSA PORRA
  #raise BaseHTTPError(response)

### Gera as datas que serão utilizadas para recuperar os dados
As datas que serão utilizadas para recuperar os dados dos charts e dos virais são geradas abaixo e armazenadas em uma lista (listDate).

In [0]:
startDate = datetime.datetime.strptime("01-01-2017", "%d-%m-%Y")
#endDate = datetime.datetime.strptime("01-02-2017", "%d-%m-%Y")
endDate   = datetime.datetime.today()

# foi retirado -1 do resultado da expressão "int ((endDate - startDate).days)" pois só existe chart para download no spotify até o dia anterior
listDate = [startDate + datetime.timedelta(n) for n in range(int ((endDate - startDate).days)-1)]
listDate[:3]

[datetime.datetime(2017, 1, 1, 0, 0),
 datetime.datetime(2017, 1, 2, 0, 0),
 datetime.datetime(2017, 1, 3, 0, 0)]

### Criação de dois data frames finais com os dados dos charts e virais

Os dois data frames finais com os dados do chart (pandao) e do viral (pandaoV) foram criados a partir da união de data frames diários gerados a cada iteração do loop de data. Foram adicionados para cada um dos dois data frames finais três atributos como forma de enriquecimento, sendo eles: dois de com as datas de referência dos dados (um com o tipo string no formato YYYY-MM-DD e o outro do tipo int no formato YYYYMMDD) e um com o track_id (gerado a partir do atributo de URL). A intenção da criação deste último é para realizar a ligação com a base de dados que contém as features de cada música.

In [0]:
# são criadas as listas que serão utilizadas para armazenar todos os data frames gerados com os dados dos charts e dos virais
spotifyCharts = []
spotifyViral = []

# criação dos data frames finais
pandao = pd.DataFrame()
pandaoV = pd.DataFrame()

# realiza um loop com todas as datas criadas a partir do startDate e endDate
for date in listDate:
  # realiza a requisição de download com concatenando a data no formato esperado
  # data frames criados a partir da função pandazize com os dados dos charts e virais diários de acordo com a data de iteração do loop
  pddia = pandazize(date, tipo='chart')
  pdviral = pandazize(date, tipo='viral')
  
  # adição da data do chart do tipo string no formato YYYY-MM-DD e do typo inteiro no formato YYYYMMDD
  pddia['data_chart'] = str(date)[:10]
  pddia['data_chart_int'] = int(date.strftime('%Y%m%d'))
  
  # adição da data do viral do tipo string no formato YYYY-MM-DD e do typo inteiro no formato YYYYMMDD
  pdviral['data_viral'] = str(date)[:10]
  pdviral['data_viral_int'] = int(date.strftime('%Y%m%d'))
  
  # adição dos data frames diários nas listas de chars e virais
  spotifyCharts.append(pddia)
  spotifyViral.append(pdviral)
  print('processado dia {}'.format(str(date)[:10]))

# para criar um único data frame foi utilizada a função concat do pandas que recebe uma lista de data frames e une tudo em único data frame
# criação de um data frame único a partir dos data frames diários de charts e do atriuto track_id que pega a string a partir do último "/" da URL
pandao = pd.concat(spotifyCharts)
pandao['track_id'] = pandao['URL'].str.slice(start=31, stop=53, step=1)

# criação de um data frame único a partir dos data frames diários de virais e do atriuto track_id que pega a string a partir do último "/" da URL
pandaoV = pd.concat(spotifyViral)
pandaoV['track_id'] = pandaoV['URL'].str.slice(start=31, stop=53, step=1)

###Conexão com Spotipy

In [0]:
client_id = "93c31efae31c4c8ea0e5ceacad67de30"
client_secret = "859960b852c04e03a1c18e34b8428b38"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

###Criação do DataFrame com Features

In [0]:
listIdTrack = pd.concat([pandao['track_id'], pandaoV['track_id']]).unique()

spotifyFeatures = []

for x in listIdTrack:
  pdtrack = sp.audio_features(x)
  spotifyFeatures.extend(pdtrack)
   
features = pd.DataFrame(spotifyFeatures)

#features.head()

### Inclusão dos data frames finais em arquivos feather
Foram utilizados arquivos de extensão .feather com o intuito de armazenar os dados em estruturas que ocupem menos espaços

In [0]:
pandao.reset_index().to_feather('pandao.feather')
pandaoV.reset_index().to_feather('pandaoV.feather')
features.reset_index().to_feather('features.feather')

### Upload dos arquivos com extensão .feather no google drive
Após a criação dos arquivos com extensão .feather que possuem os dados de chart e viral é realizado o upload dos mesmos no google drive para que sejam utilizados posteriormente, sem a necessidade de todo o processo anterior de importação dos dados seja realizado sempre que o trabalho for iniciado

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload pandao.feather file.
uploaded = drive.CreateFile({'title': 'pandao.feather'})
uploaded.SetContentFile('pandao.feather')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# Create & upload pandaoV.feather file.
uploaded2 = drive.CreateFile({'title': 'pandaoV.feather'})
uploaded2.SetContentFile('pandaoV.feather')
uploaded2.Upload()
print('Uploaded file with ID {}'.format(uploaded2.get('id')))

# Create & upload features.feather file.
uploaded3 = drive.CreateFile({'title': 'features.feather'})
uploaded3.SetContentFile('features.feather')
uploaded3.Upload()
print('Uploaded file with ID {}'.format(uploaded3.get('id')))

    100% |████████████████████████████████| 993kB 19.1MB/s 
Uploaded file with ID 1TYqs5aOkS11s09xkzoxd4Cek8PEsPJgz
Uploaded file with ID 1TCdk2m6QW5JMfNtjPQH0kDl56Y6647Q2
Uploaded file with ID 1VVZTtYDD8PEg02iqhRg4g67FSGGO5GPZ


# Data Loading

Este passo carrega os arquivos feather gerados na etapa de extração, para que a mesma não tenha que ser executada novamente

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz

charts_file_id = '15wds_Fd6lKaajtuX64fL8GRwBvh9eMzW'
viral_file_id = '1nw1Xkkmt4_sgdJyeLV1HV5oWhiB-z0k4'
features_file_id = '1RkvHxSrpANn_JzZoCw3AKnvTcfS2yYpC'

In [0]:
#define a função que salva localmente o arquivo do drive
def download_feather(file_id,filename):
  download = drive.CreateFile({'id': file_id})
  download.GetContentFile(filename)
  return None
  

In [0]:
#Realiza o download dos arquivos
download_feather(charts_file_id,'charts.feather')
download_feather(viral_file_id,'viral.feather')
download_feather(features_file_id,'features.feather')


In [0]:
#cria os dataframes com os arquivos salvos
import feather

charts = feather.read_dataframe('charts.feather')
viral = feather.read_dataframe('viral.feather')
features = feather.read_dataframe('features.feather')

In [0]:
#geracao manual de um dataframe com as datas referentes aos carnavais
#carnaval 2017: 10/02 - 14/02 (sexta à quarta-feira)
#carnaval 2018: 09/02 - 14/02 (sexta à quarta-feira)
#carnaval 2019: 01/03 - 06/03 (sexta à quarta-feira)
carnaval = pd.Series([20170210,20170211,20170212,20170213,20170214,
                      20180209,20180210,20180211,20180212,20180213,20180214,
                      20190301,20190302,20190303,20190304,20190305,20190306])


# Data Analysis

Iremos agora analisar os dados de entrada

In [0]:
import pandas_profiling
from IPython.display import display, HTML

def generate_profiling(dataframe):
  report = pandas_profiling.ProfileReport(dataframe)
  bootstrap_html = """
  <!doctype html>
    <html lang="en">
    <head>
      <meta charset="utf-8">
      <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script> 
      <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.1.1/js/bootstrap.min.js"></script>
     <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.1.1/js/bootstrap.bundle.min.js"></script>
     <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" crossorigin="anonymous">
    </head>
  </html>
  """
  display(HTML(bootstrap_html + report.html))

In [49]:
generate_profiling(features)

Number of variables,19
Number of observations,5198
Total Missing (%),0.0%
Total size in memory,771.7 KiB
Average record size in memory,152.0 B
Numeric,13
Categorical,0
Boolean,1
Date,0
Text (Unique),4
Rejected,1


In [50]:
charts.columns

Index(['index', 'Artist', 'Position', 'Streams', 'Track Name', 'URL',
       'data_chart', 'data_chart_int', 'track_id'],
      dtype='object')

In [51]:
features.columns

Index(['index', 'acousticness', 'analysis_url', 'danceability', 'duration_ms',
       'energy', 'id', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'speechiness', 'tempo', 'time_signature', 'track_href', 'type',
       'uri', 'valence'],
      dtype='object')

In [0]:
#Quais foram as musicas mais tocadas durante todos os carnavais?
hits_carnaval = pd.merge(charts,features,left_on='track_id',right_on='id')
hits_carnaval = hits_carnaval[hits_carnaval.data_chart_int.isin(carnaval)]
hits_carnaval = hits_carnaval.groupby('Track Name')['Streams'].sum()


In [63]:
hits_carnaval.shape

(422,)

In [62]:
hits_carnaval.sort_values(ascending=False)

Track Name
Vai malandra (feat. Tropkillaz & DJ Yuri Martins)                                 3741744.0
Envolvimento                                                                      3165621.0
Ta Tum Tum                                                                        2956198.0
Shape of You                                                                      2507543.0
Agora Vai Sentar                                                                  2490849.0
Vidinha de Balada - Ao Vivo                                                       2242244.0
Hear Me Now                                                                       2198989.0
Apelido Carinhoso                                                                 2173332.0
Amar Amei                                                                         2166756.0
Olha a Explosão                                                                   2151833.0
Rabiola                                                              

# Feature Engineering

# Pipeline ML